# Explore here

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Cargar el dataset
file_path = '../data/raw/diabetes.csv'
df = pd.read_csv(file_path)

# Análisis descriptivo básico
shape = df.shape
info = df.info()
describe = df.describe()

(shape, info, describe)



# Graficar histogramas para las variables numéricas
numerical_columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
df_final = df

# Seleccionar las columnas relevantes para el modelo
features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# Definir la variable objetivo
target = 'Outcome'

# Crear el dataframe final con las características seleccionadas
df_final = df[features + [target]]

# Mostrar la estructura del dataframe final
print("Estructura del dataframe final para el modelo:")
print(df_final.head())
print(f"\nDimensiones del dataframe final: {df_final.shape}")



ModuleNotFoundError: No module named 'pandas'

In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn import tree

# Definir las características (X) y la variable objetivo (y)
X = df_final.drop(columns=['Outcome'])
y = df_final['Outcome']

# Dividir el dataset en entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Crear el modelo de árbol de decisión
model = XGBClassifier(random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [6]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")
print("\nMatriz de confusión:")
print(conf_matrix)
print("\nReporte de clasificación:")
print(class_report)

Precisión del modelo: 0.69

Matriz de confusión:
[[72 27]
 [20 35]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.78      0.73      0.75        99
           1       0.56      0.64      0.60        55

    accuracy                           0.69       154
   macro avg       0.67      0.68      0.68       154
weighted avg       0.70      0.69      0.70       154



In [7]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros a probar
#param_grid = {
#    'max_depth': [3, 5, 7, 10, 15],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4],
#    'criterion': ['gini', 'entropy']
#}

# Definir los hiperparámetros 
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Entrenar usando GridSearch
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Evaluar el modelo optimizado en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred_optimized = best_model.predict(X_test)

# Evaluación del modelo optimizado
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
conf_matrix_optimized = confusion_matrix(y_test, y_pred_optimized)
class_report_optimized = classification_report(y_test, y_pred_optimized)

print(f"Precisión del modelo optimizado: {accuracy_optimized:.2f}")
print("\nMatriz de confusión del modelo optimizado:")
print(conf_matrix_optimized)
print("\nReporte de clasificación del modelo optimizado:")
print(class_report_optimized)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/home/vscode/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:53:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:53:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:53:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:53:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "criterion", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(sms

Mejores hiperparámetros:
{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
Precisión del modelo optimizado: 0.71

Matriz de confusión del modelo optimizado:
[[73 26]
 [19 36]]

Reporte de clasificación del modelo optimizado:
              precision    recall  f1-score   support

           0       0.79      0.74      0.76        99
           1       0.58      0.65      0.62        55

    accuracy                           0.71       154
   macro avg       0.69      0.70      0.69       154
weighted avg       0.72      0.71      0.71       154



In [2]:
#model.save_model("xgb_regressor_default_42.json")

from pickle import dump

# Guardar el modelo optimizado en un archivo .pkl
dump(best_model, open("xgboost_classifier_optimized.pkl", "wb"))

NameError: name 'best_model' is not defined

In [ ]:
Comparación General:
    Modelo	Accuracy	Precision (Clase 1)	Recall (Clase 1)	F1-Score (Clase 1)
    Decision Tree	0.77	0.72	0.60	0.65
    Random Forest	0.75	0.64	0.65	0.65
    XGBoost	        0.78+	0.72+	0.65+	0.66+

Con base en esta comparación, es probable que XGBoost sea la mejor opción debido a su capacidad para ofrecer mejores métricas en general, especialmente en la clase 1 (Diabetes), que es crucial en este tipo de problemas. Si has obtenido buenos resultados con XGBoost, ese sería el modelo más confiable.